# Keras 初心者ハンズオン
# はじめての画像分類

<hr>

## ページ内目次

<ul>
<li><a href="#画像ファイルの読み込み">画像ファイルの読み込み</a></li>
<li><a href="#Kerasのインポート、VGG16モデルのロード">Kerasのインポート、VGG16モデルのロード</a></li>
<li><a href="#モデルを使って画像のクラスを予測する">モデルを使って画像のクラスを予測する</a></li><li><a href="#Variableクラス中の数値データへのアクセス">Variableクラス中の数値データへのアクセス</a></li>
<li><a href="#複数画像をバッチ処理する">複数画像をバッチ処理する</a></li>
<li><a href="#model.predict()-が返す行列を理解する">model.predict()が返す行列を理解する</a></li>
<li><a href="#model.predict()-が返した確率をグラフ表示する">model.predict()が返した確率をグラフ表示する</a></li>
<li><a href="#ラベルリストの利用">ラベルリストの利用</a></li>
<li><a href="#Top-N-クラスの調査">Top-N-クラスの調査</a></li>
<li><a href="#Top-N-クラスの調査（クラスラベルとあわせて）">Top-Nクラスの調査（クラスラベルとあわせて）</a></li>
<li><a href="#matplotlib-による円グラフの表示">matplotlibによる円グラフの表示</a></li>
<li><a href="#さらなる演習">さらなる演習</a></li>
<li><a href="#様々な画像をまとめてロードする">様々な画像をまとめてロードする</a></li>
</ul>

<hr>

Keras ハンズオンにようこそ！

本ハンズオンでは、ディープラーニングや Keras フレームワークを利用したことがない方を対象として、ディープラーニングによる画像分類の方法をご紹介します。

<hr>

画像に何が映っているか、深層学習によって推測させてみましょう。 Keras を使った場合、有名なネットワーク構造がいくつか、すぐに試せる状態になっています。ここでは、2015年に発表された VGG16 を使ってみましょう。

<div style="border: 1px solid; padding: 10px">
VGG16ネットワークは、224x224の画像を1000クラスに分類するためのネットワークです。

<img src="_images/vgg16.png">
図: VGG16ネットワークの構造
<p>
Very Deep Convolutional Networks for Large-Scale Image Recognition<br>https://arxiv.org/abs/1409.1556

<p>Keras では links.model.vision.vgg.VGG16Layers クラスが定義されています。 Keras 自体に学習済みデータは提供されていませんが、 Caffe 向けに公開されている学習済みモデルを読み込んで利用できるため、学習に時間をかけなくても試せます。

https://github.com/chainer/chainer/blob/v2/chainer/links/model/vision/vgg.py

</div>

この環境には<a href="00_sample_images.ipynb">以下の画像</a>が準備されています。

<table><tr style="align: center"><td>airplane.jpg</td><td>cat.jpg</td><td>dog.jpg</td><td>dolphin.jpg</td><td>human_1.jpg</td><td>human_2.jpg</td><td>spider.jpg</td></tr>
<tr><td><img src="airplane.jpg" style="width: 96px"></td><td><img src="cat.jpg" style="width: 96px"></td><td><img src="dog.jpg"  style="width: 96px"></td><td><img src="dolphin.jpg" style="width: 96px"></td><td><img src="human_1.jpg" style="width: 96px"></td><td><img src="human_2.jpg" style="width: 96px"></td><td><img src="spider.jpg" style="width: 96px"></td></tr></table>

これらの画像から1点選択し、読み込みます。下記の Notebook は cat.jpg を選択した選定で解説します。

<hr>

## 画像ファイルの読み込み

In [ ]:
from keras.preprocessing import image
img = image.load_img("cat.jpg")

In [ ]:
print(img) # 型を見てみましょう
img

上記ブロックを実行し、猫が表示されましたか？

<hr>

## Kerasのインポート、VGG16モデルのロード

Kerasのインポート、訓練済みモデルのロード

In [ ]:
from keras.applications import vgg16
model = vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

ここで、 model には、 VGG16 と呼ばれる有名な画像分類ネットワークがロードされます。

<div style="border: 1px solid; padding: 10px">
<tt>In[*]</tt> という表示がしばらく続くかもしれません。VGG16のデータファイルがない場合、インターネットからデータ(約500MB)をダウンロードし変換する必要があるため、辛抱強く待ってみてください。</div>

モデルの確認

In [ ]:
model

model オブジェクトが keras.engine.training.Model クラスであることを確認してみてください。

<hr>

## モデルを使って画像のクラスを予測する

画像 img をニューラルネットワークに入力し、出力 p を得る

<div style="border: 1px solid; padding: 10px">model.predict() は、２次元画像の配列を受け取ります。</div>

In [ ]:
import numpy as np
from keras.preprocessing import image
# 画像データをVGG16用に編集
img = img.resize((256,256)).crop((16,16,240,240))
# 画像データを行列式へ変形
d3_mtx = image.img_to_array(img)        # 3D-tensor(rows, cols, channels)
d4_mtx = np.expand_dims(d3_mtx, axis=0) # 4D-tensor(samples, rows, cols, channels)

d4_mtx = vgg16.preprocess_input(d4_mtx)   # 全体-平均値＆RGB->BGR
p = model.predict(d4_mtx, verbose=1)


※ model.predict() の実行には、いくらか時間がかかります。



予測結果（の生データ）を表示してみる

In [ ]:
p

たくさんの値が入った行列と型情報のTupleが見えるはずです。

In [ ]:
p.shape

<hr>

## 数値データへのアクセス

kerasの予測結果は、chainerのVariableクラスと違って直接numpyのndarrayクラスに格納されます。

In [ ]:
p.__class__ # => numpy.ndarrayクラス

In [ ]:
p[0, 1] # => (0, 1)の要素（numpy.float32クラス）

<hr>

## 複数画像をバッチ処理する

複数の画像を連続で処理したい場合は、配列として画像を複数渡して、まとめて予測を得ることができます。この際、 model.predict() が返す配列の要素数は（イメージ数、 1000クラス）になります。

1枚単位で画像を予測する場合と結果は一緒ですが、GPU搭載サーバーで短時間に多数の画像を予測したい場合、多数の画像をまとめて predict() することにより、システム全体のスループットが向上します。


### 1枚のみ処理する場合

In [ ]:
d3_mtx = image.img_to_array(img)
d4_mtx = np.expand_dims(d3_mtx, axis=0)
p1 = model.predict(d4_mtx, verbose=1)

In [ ]:
p1.shape # => (1, 1000)

### 2枚処理する場合　

In [ ]:
d4_mtces = d4_mtx
d4_mtces = np.append(d4_mtces, d4_mtx, axis=0)
p2 = model.predict(d4_mtces, verbose=1)

In [ ]:
p2.shape # 演習: 結果を予想してください

### 3枚処理する場合

【演習】<br><br>
３枚の画像を同時に model.predict() に渡して、期待どおりの要素数の配列が戻されることを確認してください。  
個人マシンで行う場合は、メモリの使用量に気を付けてください。

In [ ]:
# 演習：　３枚の画像を同時に model.predict() に渡してください
d4_mtces = d4_mtx
d4_mtces = np.append(d4_mtces, d4_mtx, axis=0)
d4_mtces = np.append(d4_mtces, d4_mtx, axis=0)
p3 = model.predict(d4_mtces)
p3.shape

<hr>

## model.predict() が返す行列を理解する

Python での配列操作に自身がない方は下記をお試しください。

In [ ]:
import numpy as np

a = np.random.random((10,10)) # 10行×10桁の配列を作成
a

In [ ]:
# 最初の行 (0行目)の10桁を返す
a[1]

In [ ]:
# 最初の行 (0行目)の10桁から、4つめの要素を返す
a[1][4]

In [ ]:
# 下記のとおり表記できる
a[1, 4]

In [ ]:
# 便利情報: 各行の4列目だけを取得する
a[:,4]

<hr>

## model.predict() が返した確率をグラフ表示する

<div style="border: 1px solid; padding: 10px">以降、予測した最初の画像 (0行目) の内容について調査していきます。変数 d に p.data[0] を代入し、以降の操作は d に対して行います。</div>

In [ ]:
# 以後、 p.data[0] にとしてアクセスする
d = p[0]

In [ ]:
d.shape # dの要素数を確認したい方はお試しください

In [ ]:
d # 中身を見たい方はお試しください

d を棒グラフとして表示してみます。

In [ ]:
import matplotlib.pyplot as plt
 
%matplotlib inline 
plt.style.use('ggplot')
plt.figure(figsize=(20,10))
plt.bar(left=range(len(d)), height=d)
plt.plot()

<div style="border: 1px solid; padding: 10px">これは、0番目から999番目のクラスまで、そのクラスである確率の棒グラフです。200 ~ 400 の間にもっとも高いバーが見えているはずですが、この最も高い値がいくつめの要素か調べていきます。</div>

配列 d の中から最大値を探す


In [ ]:
import numpy as np

predicted_label = np.argmax(d)

最大値がある要素の番号を表示


In [ ]:
predicted_label

# d[predicted_label] は d の中の最大値で、確率（0.0〜1.0）


In [ ]:
d[predicted_label]

np.argmax(d) が 285  を返した場合、入力した画像 img は 285 番目のクラスに分類されたことを意味しています。

<hr>

## ラベルリストの利用

入力した画像は、 predicted_label 番目（285番目）のクラスである確率が高いことがわかりました。しかし、この 285 番目のクラスというのは何を指しているのでしょうか。

（このモデルの学習時に利用された）クラスラベルリスト <a href="synset_words.txt">synset_words.txt</a>  と照合することで、このクラスラベルが何を意味しているのかを調べます。

In [ ]:
labels = open("synset_words.txt", "r").readlines()

In [ ]:
# labels の内容を確認する (1000個まで表示すると長いので、とりあえず最初の10個まで)
labels[:10]

入力画像に対して最も高い確率を示したクラスのラベル文字列を表示


In [ ]:
# predict_label 番目の要素を調べる
labels[predicted_label]

実行した結果、 Egyptian cat 、つまり、ニューラルネットワークは、この画像が Egyptian cat に分類した、ということになります。

<hr>

## Top-N クラスの調査

<div style="border: 1px solid; padding: 10px">返された値は「入力された画像が各クラスの値である確率」だと説明しました。また、最大値は 285番目の要素であることがわかりました。最大値に続く、大きな値を探してみましょう。</div>

確率を昇順にソートして、最後の最大3件を表示


In [ ]:
sorted(d)[-3:]

確率を降順にソートして、最初の最大3件を表示（上記とは順序のみの違い）


In [ ]:
sorted(d, reverse=True)[:3]

上位3クラスが占める確率


In [ ]:
np.sum(sorted(d, reverse=True)[:3])

<hr>

## Top-N クラスの調査（クラスラベルとあわせて）

<div style="border: 1px solid; padding: 10px">このままではクラスラベルがなく、確率値が何に対応しているのかがわかりません。以下の操作で確率とクラスラベルをマージした上でソートしてみましょう。</div>

確率とクラスラベルを組みわせたリストの作成

In [ ]:
p2 = list(zip(p[0], labels))

作成した組み合わせリストから Top 3 を表示

In [ ]:
sorted(p2, reverse=True)[:3]

<div style="border: 1px solid; padding: 10px">
出力例

<p>
[<br>(0.52825135, 'n02124075 Egyptian cat\n'), <br>
   (0.11501167, 'n02123045 tabby, tabby cat\n'), <br>
   (0.052949127, 'n02123159 tiger cat\n')<br>]
</div>

<hr>

【演習】<br><br>同様に、Top-10 クラスを確認してください。

In [ ]:
p2 = list(zip(p[0], labels))
sorted(p2, reverse=True)[:10]

<hr>

## matplotlib による円グラフの表示

<div style="border: 1px solid; padding: 10px">降順にソートした確率一覧を matplotlib の円グラフにすることで、Top-Nクラスがどれぐらいの割合を占めるのかを視覚的に確認できます。</div>

In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt

d_sorted = sorted(d, key=lambda x:-x)
plt.style.use('ggplot')
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(20,20))
plt.pie(d_sorted)
plt.plot()

<hr>

## さらなる演習

【演習】<br>

* ハンズオン環境に用意されている画像を model.predict() にわたし、各画像の予測クラス（１画像あたり1クラス）を表示してください。


<a href="00_sample_images.ipynb">ハンズオン環境に存在する画像</a>をまとめて読み込みたい場合は、次のブロックを実行してください。

In [ ]:
img_airplane = image.load_img("airplane.jpg")
img_cat = image.load_img("cat.jpg")
img_dog = image.load_img("dog.jpg")
img_dolphin = image.load_img("dolphin.jpg")
img_human1 = image.load_img("human_1.jpg")
img_human2 = image.load_img("human_2.jpg")
img_spider = image.load_img("spider.jpg")

In [ ]:
# 演習
img_airplane = img_airplane.resize((256,256)).crop((16,16,240,240))
d3_air_mtx = image.img_to_array(img_airplane)
d4_air_mtx = np.expand_dims(d3_air_mtx, axis=0)

p = model.predict(d4_air_mtx, verbose=1)